In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SaveMode

val spark = SparkSession.builder().master("spark://spark:7077").appName("Financial_csv").getOrCreate()

spark = org.apache.spark.sql.SparkSession@63b82589


org.apache.spark.sql.SparkSession@63b82589

In [3]:
val f_csv=spark.read.option("header", "true").option("multiLine", true).csv("../data/Financial_Sample.csv")//读取文件

lastException = null


java.lang.InterruptedException: None

In [3]:
val fcol = f_csv.columns

")


In [5]:
def col2Float(S: String) = {
    var S_deal = S.trim()
    if (S_deal.charAt(0).equals('$')){
    S_deal = S_deal.substring(1, S_deal.length)
    }
    S_deal = S_deal.trim()
    if (S_deal.indexOf("(") != -1){
        S_deal = S_deal.substring(S_deal.indexOf("(")+1, S_deal.indexOf(")"))
    }
    if (S_deal.charAt(0).equals('-')){
    "0.0"
    }
    else{
        S_deal.replace(",","").toFloat.formatted("%.2f")
    }
    
}
var c2f = udf(col2Float _)

c2f = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


col2Float: (S: String)String


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [68]:
col2Float("$(453,375.00)")

453375.00

In [4]:
def columnName_deal(S: String)={
    var S_deal = S.trim
    var a = S_deal.indexOf("(")
    var b = S_deal.indexOf("（")
    if (a != -1){
        if (b<a && b != -1)
        {
            S_deal = S_deal.substring(0,b).trim
        }
        else{
            S_deal = S_deal.substring(0,a).trim
        }  
    }
    else{
        if (b != -1)
        {
            S_deal = S_deal.substring(0,b).trim
        }
        else{
            S_deal = S_deal.trim
        }
    }
    S_deal.replace(" ", "_")
}

columnName_deal: (S: String)String


In [6]:
val str_trim = udf((str: String) => str.trim())

str_trim = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [7]:
def date_deal(S: String)={
    S.trim.replace("/","-")
}
var dd = udf(date_deal _)

dd = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


date_deal: (S: String)String


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [8]:
var newf_csv = f_csv.toDF(fcol.map(columnName_deal(_)):_*)

newf_csv = [Segment: string, Country: string ... 14 more fields]


[Segment: string, Country: string ... 14 more fields]

In [9]:
newf_csv = newf_csv.withColumn("Date", dd(col("Date")))
                   .withColumn("Month_Number", str_trim(col("Month_Number")))
                   .withColumn("Year", str_trim(col("Year")))

newf_csv = [Segment: string, Country: string ... 14 more fields]


[Segment: string, Country: string ... 14 more fields]

In [10]:
val newfcol = newf_csv.columns

newfcol = Array(Segment, Country, Product, Discount_Band, Units_Sold, Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit, Date, Month_Number, Month_Name, Year)


Array(Segment, Country, Product, Discount_Band, Units_Sold, Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit, Date, Month_Number, Month_Name, Year)

In [11]:
newfcol.map(column =>{
    newf_csv = newf_csv.withColumn(column, str_trim(col(column)))
})

Array((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ())

In [12]:
val float_col = newfcol.slice(5,12)

float_col = Array(Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit)


Array(Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit)

In [28]:
float_col.toSeq.foldLeft("")(_ + "," + _).substring(1)

Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit

In [ ]:
float_col.map(column =>{
    newf_csv = newf_csv.withColumn(column, c2f(col(column)))
})

In [16]:
newf_csv.select("Gross_Sales").collect()

Array([$32,370.00], [$26,420.00], [$32,670.00], [$13,320.00], [$37,050.00], [$529,550.00], [$13,815.00], [$30,216.00], [$37,980.00], [$18,540.00], [$37,050.00], [$333,187.50], [$287,400.00], [$15,022.00], [$43,125.00], [$9,225.00], [$5,840.00], [$14,610.00], [$30,216.00], [$352,100.00], [$4,404.00], [$6,181.00], [$8,235.00], [$236,400.00], [$37,080.00], [$8,001.00], [$603,750.00], [$10,944.00], [$32,280.00], [$36,340.00], [$529,550.00], [$10,451.00], [$225,500.00], [$25,932.00], [$352,100.00], [$18,540.00], [$352,625.00], [$43,125.00], [$600,300.00], [$34,056.00], [$32,670.00], [$13,320.00], [$534,450.00], [$645,300.00], [$36,340.00], [$962,500.00], [$23,436.00], [$527,437.50], [$37,980.00], [$11,802.00], [$25,692.00], [$8,001.00], [$9,225.00], [...

In [13]:
newf_csv.write.mode(SaveMode.Append).partitionBy("Year", "Month_Number", "Date").saveAsTable("financial")
//.format("hive")

In [14]:
var pre_fcsv = spark.sql("select Country, Product, Units_Sold, Sales, COGS, Profit, Year, Month_Number as Month from financial ")

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [15]:
pre_fcsv.show

+--------------------+---------+----------+-----------+-----------+-----------+----+-----+
|             Country|  Product|Units_Sold|      Sales|       COGS|     Profit|Year|Month|
+--------------------+---------+----------+-----------+-----------+-----------+----+-----+
|United States of ...|    Paseo|      1143|  $8,001.00|  $5,715.00|  $2,286.00|2014|   10|
|United States of ...| Amarilla|      1143|  $8,001.00|  $5,715.00|  $2,286.00|2014|   10|
|              Mexico|Carretera|      1397|$484,060.50|$363,220.00|$120,840.50|2014|   10|
|              Mexico|      VTT|      1397|$484,060.50|$363,220.00|$120,840.50|2014|   10|
|              Canada|Carretera|      1295| $15,229.20|  $3,885.00| $11,344.20|2014|   10|
|United States of ...|  Montana|      1566| $30,693.60| $15,660.00| $15,033.60|2014|   10|
|              Canada|    Paseo|      1295| $15,229.20|  $3,885.00| $11,344.20|2014|   10|
|United States of ...|     Velo|      1566| $30,693.60| $15,660.00| $15,033.60|2014|   10|

In [61]:
pre_fcsv.select("Sales").show

+-----------+
|      Sales|
+-----------+
|  $8,001.00|
|  $8,001.00|
|$484,060.50|
|$484,060.50|
| $15,229.20|
| $30,693.60|
| $15,229.20|
| $30,693.60|
|$986,811.00|
|$986,811.00|
|$243,591.25|
|$243,591.25|
| $29,246.40|
| $29,246.40|
|$582,048.00|
|$582,048.00|
|$731,472.00|
|$731,472.00|
| $17,881.85|
| $17,881.85|
+-----------+
only showing top 20 rows



In [16]:
pre_fcsv = pre_fcsv.withColumn("Sales", c2f(col("Sales")))

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [17]:
pre_fcsv = pre_fcsv.withColumn("COGS", c2f(col("COGS")))
pre_fcsv.show

+--------------------+---------+----------+---------+---------+-----------+----+-----+
|             Country|  Product|Units_Sold|    Sales|     COGS|     Profit|Year|Month|
+--------------------+---------+----------+---------+---------+-----------+----+-----+
|United States of ...|    Paseo|      1143|  8001.00|  5715.00|  $2,286.00|2014|   10|
|United States of ...| Amarilla|      1143|  8001.00|  5715.00|  $2,286.00|2014|   10|
|              Mexico|Carretera|      1397|484060.50|363220.00|$120,840.50|2014|   10|
|              Mexico|      VTT|      1397|484060.50|363220.00|$120,840.50|2014|   10|
|              Canada|Carretera|      1295| 15229.20|  3885.00| $11,344.20|2014|   10|
|United States of ...|  Montana|      1566| 30693.60| 15660.00| $15,033.60|2014|   10|
|              Canada|    Paseo|      1295| 15229.20|  3885.00| $11,344.20|2014|   10|
|United States of ...|     Velo|      1566| 30693.60| 15660.00| $15,033.60|2014|   10|
|             Germany|     Velo|      2877|

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [18]:
pre_fcsv = pre_fcsv.withColumn("Profit", c2f(col("Profit")))
pre_fcsv.show

+--------------------+---------+----------+---------+---------+---------+----+-----+
|             Country|  Product|Units_Sold|    Sales|     COGS|   Profit|Year|Month|
+--------------------+---------+----------+---------+---------+---------+----+-----+
|United States of ...|    Paseo|      1143|  8001.00|  5715.00|  2286.00|2014|   10|
|United States of ...| Amarilla|      1143|  8001.00|  5715.00|  2286.00|2014|   10|
|              Mexico|Carretera|      1397|484060.50|363220.00|120840.50|2014|   10|
|              Mexico|      VTT|      1397|484060.50|363220.00|120840.50|2014|   10|
|              Canada|Carretera|      1295| 15229.20|  3885.00| 11344.20|2014|   10|
|United States of ...|  Montana|      1566| 30693.60| 15660.00| 15033.60|2014|   10|
|              Canada|    Paseo|      1295| 15229.20|  3885.00| 11344.20|2014|   10|
|United States of ...|     Velo|      1566| 30693.60| 15660.00| 15033.60|2014|   10|
|             Germany|     Velo|      2877|986811.00|748020.00|23

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [19]:
pre_fcsv = pre_fcsv.withColumn("YM", concat_ws("-",col("Year"), col("Month"))).drop("Year","Month")

pre_fcsv = [Country: string, Product: string ... 5 more fields]


[Country: string, Product: string ... 5 more fields]

In [41]:
val float_2f = udf((str: String) => str.toFloat.formatted("%.2f"))

float_2f = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


lastException: Throwable = null


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

# 月度销售额top3国家

In [23]:
var top3_sales_country = 
pre_fcsv.groupBy("Country", "YM")
        .agg(sum("Sales") as "Sales_c")
top3_sales_country.show

+--------------------+-------+------------------+
|             Country|     YM|           Sales_c|
+--------------------+-------+------------------+
|              France| 2013-9| 821599.5900000001|
|              Mexico| 2014-1|1655822.8499999999|
|              France| 2014-3|        1559748.75|
|              Mexico|2014-10|1855574.2999999998|
|United States of ...|2013-12| 662615.5800000001|
|              Mexico|2013-10|        1166052.56|
|              France| 2014-4|         1332862.7|
|              France| 2014-8|         779802.09|
|             Germany|2013-12| 857856.8500000001|
|              Mexico|2014-11|1123522.8399999999|
|             Germany|2014-11|          617106.5|
|              Canada| 2014-2|1482165.9799999997|
|              Canada|2014-11|         952833.26|
|              Mexico| 2014-9|1022441.2599999999|
|United States of ...| 2014-4|        1616624.48|
|             Germany| 2014-9|1255161.9000000001|
|             Germany| 2014-3|479509.58999999997|


top3_sales_country = [Country: string, YM: string ... 1 more field]


[Country: string, YM: string ... 1 more field]

In [25]:
import org.apache.spark.sql.expressions.Window

In [31]:
val win1 = Window.partitionBy("YM").orderBy(- col("Sales_c"))

win1 = org.apache.spark.sql.expressions.WindowSpec@2b98ddc8


org.apache.spark.sql.expressions.WindowSpec@2b98ddc8

In [42]:
val c_sql1 = top3_sales_country.withColumn("top", rank() over win1)
                    .filter("top <= 3").withColumn("Sales_c", float_2f(col("Sales_c")))
c_sql1.show

+--------------------+-------+----------+---+
|             Country|     YM|   Sales_c|top|
+--------------------+-------+----------+---+
|              Mexico| 2014-1|1655822.88|  1|
|              France| 2014-1|1544720.75|  2|
|United States of ...| 2014-1|1346026.50|  3|
|              Canada|2013-12|1587259.25|  1|
|              Mexico|2013-12|1325568.13|  2|
|              France|2013-12| 935141.31|  3|
|United States of ...| 2014-7|2308798.25|  1|
|              Canada| 2014-7|2109549.25|  2|
|             Germany| 2014-7|1609549.75|  3|
|             Germany| 2013-9|1095488.13|  1|
|United States of ...| 2013-9| 981022.88|  2|
|              Canada| 2013-9| 939195.19|  3|
|United States of ...| 2014-4|1616624.50|  1|
|              Canada| 2014-4|1593563.00|  2|
|             Germany| 2014-4|1394813.50|  3|
|              Mexico| 2014-2|1597700.38|  1|
|              France| 2014-2|1537438.50|  2|
|              Canada| 2014-2|1482166.00|  3|
|              Canada| 2014-6|2725

c_sql1 = [Country: string, YM: string ... 2 more fields]


[Country: string, YM: string ... 2 more fields]

# 月度利润率top3国家

In [44]:
var top3_profitraio_country = 
pre_fcsv.groupBy("Country", "YM")
        .agg(sum("COGS") as "COGS_c", sum("Profit") as "Profit_c")
        .withColumn("Profit_ratio", col("Profit_c")/col("COGS_c"))

top3_profitraio_country = [Country: string, YM: string ... 3 more fields]


[Country: string, YM: string ... 3 more fields]

In [45]:
val win2 = Window.partitionBy("YM").orderBy(- col("Profit_ratio"))

win2 = org.apache.spark.sql.expressions.WindowSpec@4df400ca


org.apache.spark.sql.expressions.WindowSpec@4df400ca

In [48]:
val c_sql2 = top3_profitraio_country.withColumn("top", rank() over win2)
                    .filter("top <= 3").select(col("Country"), col("YM"), float_2f(col("Profit_ratio")), col("top"))//.withColumn("Profit_ratio", float_2f(col("Sales_c")))
c_sql2.show

+--------------------+-------+-----------------+---+
|             Country|     YM|UDF(Profit_ratio)|top|
+--------------------+-------+-----------------+---+
|              France| 2014-1|             0.20|  1|
|              Mexico| 2014-1|             0.18|  2|
|              Canada| 2014-1|             0.17|  3|
|              Mexico|2013-12|             0.22|  1|
|             Germany|2013-12|             0.20|  2|
|              Canada|2013-12|             0.20|  3|
|United States of ...| 2014-7|             0.16|  1|
|              Canada| 2014-7|             0.14|  2|
|              France| 2014-7|             0.13|  3|
|              France| 2013-9|             0.31|  1|
|              Mexico| 2013-9|             0.24|  2|
|             Germany| 2013-9|             0.23|  3|
|              Mexico| 2014-4|             0.20|  1|
|United States of ...| 2014-4|             0.18|  2|
|              Canada| 2014-4|             0.18|  3|
|              France| 2014-2|             0.2

c_sql2 = [Country: string, YM: string ... 2 more fields]


[Country: string, YM: string ... 2 more fields]

# 月度畅销top2商品

In [49]:
var top2_sold_product = 
pre_fcsv.groupBy("Product", "YM")
        .agg(sum("Units_Sold") as "Units_Sold_p")

top2_sold_product = [Product: string, YM: string ... 1 more field]


[Product: string, YM: string ... 1 more field]

In [50]:
val win3 = Window.partitionBy("YM").orderBy(- col("Units_Sold_p"))

win3 = org.apache.spark.sql.expressions.WindowSpec@3c94fcb9


org.apache.spark.sql.expressions.WindowSpec@3c94fcb9

In [54]:
val p_sql1 = top2_sold_product.withColumn("top", rank() over win3)
                    .filter("top <= 2")
p_sql1.show

+---------+-------+------------+---+
|  Product|     YM|Units_Sold_p|top|
+---------+-------+------------+---+
|    Paseo| 2014-1|     23598.0|  1|
| Amarilla| 2014-1|     10019.5|  2|
|    Paseo|2013-12|     18856.0|  1|
|      VTT|2013-12|      8601.0|  2|
|    Paseo| 2014-7|     22637.5|  1|
| Amarilla| 2014-7|     11620.5|  2|
|    Paseo| 2013-9|     15790.0|  1|
|      VTT| 2013-9|      7953.0|  2|
|    Paseo| 2014-4|     23653.0|  1|
|Carretera| 2014-4|     13590.5|  2|
|    Paseo| 2014-2|     14629.0|  1|
|      VTT| 2014-2|     11210.0|  2|
|    Paseo| 2014-6|     25598.0|  1|
|      VTT| 2014-6|     17642.0|  2|
|    Paseo|2013-11|     20812.0|  1|
|      VTT|2013-11|     13256.0|  2|
|    Paseo|2014-11|     14129.0|  1|
|  Montana|2014-11|     10418.0|  2|
|    Paseo| 2014-9|     17416.0|  1|
| Amarilla| 2014-9|      9240.0|  2|
+---------+-------+------------+---+
only showing top 20 rows



p_sql1 = [Product: string, YM: string ... 2 more fields]


[Product: string, YM: string ... 2 more fields]

# 月度利润额最低top2商品

In [53]:
var top2_sales_loe_product = 
pre_fcsv.groupBy("Product", "YM")
        .agg(sum("Profit") as "Profit_p")

top2_sales_loe_product = [Product: string, YM: string ... 1 more field]


[Product: string, YM: string ... 1 more field]

In [55]:
val win4 = Window.partitionBy("YM").orderBy(col("Profit_p"))

win4 = org.apache.spark.sql.expressions.WindowSpec@57a77348


org.apache.spark.sql.expressions.WindowSpec@57a77348

In [58]:
val p_sql2 = top2_sales_loe_product.withColumn("top", rank() over win4)
                    .filter("top <= 2").withColumn("Profit_p", float_2f(col("Profit_p")))
p_sql2.show

+---------+-------+---------+---+
|  Product|     YM| Profit_p|top|
+---------+-------+---------+---+
|     Velo| 2014-1| 33366.28|  1|
|Carretera| 2014-1| 58392.40|  2|
|Carretera|2013-12| 56440.12|  1|
|  Montana|2013-12| 64993.07|  2|
|      VTT| 2014-7| 25720.00|  1|
| Amarilla| 2014-7| 26787.55|  2|
| Amarilla| 2013-9| 50549.23|  1|
|     Velo| 2013-9| 57200.65|  2|
| Amarilla| 2014-4| 66134.40|  1|
|Carretera| 2014-4| 75822.68|  2|
|  Montana| 2014-2|  7220.82|  1|
|     Velo| 2014-2| 38983.75|  2|
|      VTT| 2014-6| 67657.06|  1|
|  Montana| 2014-6|115545.32|  2|
|Carretera|2013-11| 14636.02|  1|
| Amarilla|2013-11| 18673.95|  2|
|     Velo|2014-11|  9853.72|  1|
|Carretera|2014-11| 35039.25|  2|
|  Montana| 2014-9| 11495.12|  1|
|     Velo| 2014-9| 45643.75|  2|
+---------+-------+---------+---+
only showing top 20 rows



p_sql2 = [Product: string, YM: string ... 2 more fields]


[Product: string, YM: string ... 2 more fields]